In [5]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

from tensorflow.keras.datasets import mnist

In [3]:
num_classes = 10
num_features = 784

learning_rate = 0.001
training_steps = 1000
batch_size = 32
display_step = 100

num_input = 28
timesteps = 28
num_units = 32

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, 28, 28])

x_train, x_test = x_train / 255., x_test / 255.

In [21]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [22]:
class LSTM(Model):

    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = layers.LSTM(units=num_units)
        self.out = layers.Dense(num_classes)

    def call(self, x, training=False):
        x = self.lstm(x)
        x = self.out(x)

        if not training:
            x = tf.nn.softmax(x)
        return x

lstm_net = LSTM()


In [26]:
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

optimizer = tf.optimizers.Adam(learning_rate)

In [27]:
def run_optimization(x, y):

    with tf.GradientTape() as tape:

        pred = lstm_net(x)

        loss = cross_entropy_loss(pred, y)

    trainable_variables = lstm_net.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [29]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % 50 == 0:
        pred = lstm_net(batch_x, training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 0.091127, accuracy: 0.968750
step: 100, loss: 0.553700, accuracy: 0.875000
step: 150, loss: 0.472050, accuracy: 0.843750
step: 200, loss: 0.757861, accuracy: 0.843750
step: 250, loss: 0.400416, accuracy: 0.875000
step: 300, loss: 0.336105, accuracy: 0.906250
step: 350, loss: 0.360020, accuracy: 0.937500
step: 400, loss: 0.752471, accuracy: 0.906250
step: 450, loss: 0.815834, accuracy: 0.781250
step: 500, loss: 0.035491, accuracy: 1.000000
step: 550, loss: 0.791227, accuracy: 0.843750
step: 600, loss: 0.564524, accuracy: 0.906250
step: 650, loss: 0.558247, accuracy: 0.937500
step: 700, loss: 0.870756, accuracy: 0.843750
step: 750, loss: 0.380751, accuracy: 0.937500
step: 800, loss: 0.364440, accuracy: 0.906250
step: 850, loss: 0.022729, accuracy: 1.000000
step: 900, loss: 0.172708, accuracy: 0.968750
step: 950, loss: 0.519006, accuracy: 0.937500
step: 1000, loss: 0.590811, accuracy: 0.906250
